In [ ]:
import  pandas as pd 

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from catboost import CatBoostClassifier

import shap

import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

